<a href="https://colab.research.google.com/github/PyMalama/spare-parts-reconciliation/blob/main/spare_parts_reconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 380.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 71.4 MB/s eta 0:00:00


In [ ]:
!pip install pytesseract

In [ ]:
!pip install pdf2image

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (273 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import fitz  # PyMuPDF (Efficient text extraction)
import pytesseract
from pdf2image import convert_from_path  # OCR (fallback)
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
import glob
from difflib import SequenceMatcher

#  PARAMETERS
pdf_paths = glob.glob("/content/*.pdf")  # Load all PDFs in directory
excel_path = "/content/Updated_Spare_Parts 7.xlsx"
output_path = "/content/Updated_Spare_Parts.xlsx"
machine_name = "PACKER 32"
similarity_threshold = 0.7  # 70% similarity required for matching

# Load Excel file (keeps styles, fonts, colors)
wb = load_workbook(excel_path)
ws = wb["Spares"]

# Find column indexes
header_row = [cell.value for cell in ws[1]]
part_number_col = header_row.index("Part Number") + 1
machine_name_col = header_row.index("Machine Name") + 1
spare_part_col = header_row.index("Spare Part") + 1

# Normalize part numbers (remove prefixes & non-numeric characters)
def normalize_part_number(part):
    if not part:
        return ""
    part = part.upper().strip()
    part = re.sub(r"^[A-C]-", "", part)  # Remove "A-", "B-", "C-" prefix
    part = re.sub(r"[^\d\s-]", "", part)  # Keep only numbers, spaces, and dashes

    return part

# Extract text efficiently using PyMuPDF (fitz)
def extract_text_pymupdf(pdf_path):
    text = []
    doc = fitz.open(pdf_path)
    for page in doc:  # Process page by page
        text.append(page.get_text("text"))
    return "\n".join(filter(None, text))

# OCR extraction (only if PyMuPDF fails)
def extract_text_ocr(pdf_path):
    text = []
    images = convert_from_path(pdf_path)
    for img in images:
        text.append(pytesseract.image_to_string(img))
    return "\n".join(text)

# Extract and normalize part numbers from text
import re

def extract_part_numbers(text):
    raw_parts = re.findall(r"\b[A-Z\d]-?\d{2,10}(?:\s\d{2,10})*\b", text)  # Supports numbers with spaces
    return {normalize_part_number(part) for part in raw_parts}


# Extract part numbers from manuals
manual_part_numbers = set()
for pdf_path in pdf_paths:
    print(f"\n Processing: {pdf_path}")

    # Extract text using PyMuPDF & OCR (combining both)
    extracted_text = extract_text_pymupdf(pdf_path)
    ocr_text = extract_text_ocr(pdf_path)
    extracted_text = f"{extracted_text}\n{ocr_text}"

    # Extract & normalize part numbers
    extracted_parts = extract_part_numbers(extracted_text)
    manual_part_numbers.update(extracted_parts)

#  Debugging: Print extracted part numbers
print(f"\n Extracted {len(manual_part_numbers)} normalized part numbers.")
print(" Sample extracted parts:", list(manual_part_numbers)[:10])

# Load spreadsheet part numbers
spreadsheet_part_numbers = set()
spreadsheet_data = {}

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    part_number = normalize_part_number(str(row[part_number_col - 1].value))
    spare_part = str(row[spare_part_col - 1].value).strip()

    spreadsheet_part_numbers.add(part_number)
    spreadsheet_data[part_number] = spare_part

#  Debugging: Print extracted spreadsheet parts
print(f"\n Extracted {len(spreadsheet_part_numbers)} spreadsheet part numbers.")
print(" Sample spreadsheet parts:", list(spreadsheet_part_numbers)[:10])

# Identify unmatched part numbers
supersedes_matches = {part for part in spreadsheet_part_numbers if part not in manual_part_numbers}

# Similarity matching for part numbers & descriptions
def similar_part_numbers(p1, p2):
    return SequenceMatcher(None, p1, p2).ratio()

def similar_spare_parts(sp1, sp2):
    return SequenceMatcher(None, sp1, sp2).ratio()

final_supersedes = {}
for sup in supersedes_matches:
    for part in manual_part_numbers:
        if similar_part_numbers(sup, part) >= similarity_threshold:
            spare_desc1 = spreadsheet_data.get(sup, "")
            spare_desc2 = spreadsheet_data.get(part, "")

            if similar_spare_parts(spare_desc1, spare_desc2) >= 0.6:
                final_supersedes[sup] = part

# Print & update matched machine names
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")

if final_supersedes:
    print("\n Highly Similar Superseded Part Numbers Found:")
    for new, old in final_supersedes.items():
        print(f" {new} → Likely Supersedes → {old}")

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        part_number = normalize_part_number(str(row[part_number_col - 1].value))
        machine_cell = row[machine_name_col - 1]

        if part_number in manual_part_numbers or part_number in final_supersedes:
            machine_cell.value = f"{machine_cell.value} & {machine_name}" if machine_cell.value else machine_name
            machine_cell.fill = red_fill  # Highlight in red

        if part_number in final_supersedes:
            row[part_number_col - 1].fill = green_fill  # Highlight superseded part numbers in green

# Save the updated workbook
wb.save(output_path)

print(f"\n Updated Excel file saved as: {output_path}")



 Processing: /content/SPC-3025060-0110.pdf


In [ ]:
import fitz  # PyMuPDF (Efficient text extraction)
import pytesseract
from pdf2image import convert_from_path  # OCR (fallback)
import pandas as pd
import re
from openpyxl import load_workbook
import glob
from difflib import get_close_matches

#  PARAMETERS
pdf_paths = glob.glob("/content/*.pdf")  # Load all PDFs in directory
excel_path = "/content/Zambia Spares allocation- 21-03-25.xlsx"
output_path = "/content/Updated_Spare_Parts.xlsx"
machine_name = "PACKER 32"
similarity_threshold = 0.7  # 70% similarity required for matching

# Load Excel file (keeps styles, fonts, colors)
wb = load_workbook(excel_path)
ws = wb["Spares"]

# Find column indexes
header_row = [cell.value for cell in ws[1]]
part_number_col = header_row.index("Part Number") + 1
machine_name_col = header_row.index("Machine Name") + 1
spare_part_col = header_row.index("Spare Part") + 1

# Normalize part numbers
def normalize_part_number(part):
    if not part:
        return ""
    part = part.upper().strip()
    part = re.sub(r"^[A-C]-", "", part)  # Remove "A-", "B-", "C-" prefix
    part = re.sub(r"[^\d-]", "", part)  # Keep only numbers and dashes
    return part

# Extract text efficiently using PyMuPDF (fitz)
def extract_text_pymupdf(pdf_path):
    return "\n".join([page.get_text("text") for page in fitz.open(pdf_path)])

# OCR extraction (only if PyMuPDF fails)
def extract_text_ocr(pdf_path):
    return "\n".join([pytesseract.image_to_string(img) for img in convert_from_path(pdf_path)])

# Extract and normalize part numbers from text
def extract_part_numbers(text):
    raw_parts = re.findall(r"[A-Z]?-?\d{5,10}-?\d{1,4}", text)
    return {normalize_part_number(part) for part in raw_parts}

# Extract part numbers from manuals
manual_part_numbers = set()
for pdf_path in pdf_paths:
    print(f"\n Processing: {pdf_path}")
    extracted_text = extract_text_pymupdf(pdf_path) or extract_text_ocr(pdf_path)
    manual_part_numbers.update(extract_part_numbers(extracted_text))

print(f"\n Extracted {len(manual_part_numbers)} normalized part numbers.")
print(" Sample extracted parts:", list(manual_part_numbers)[:10])

# Load spreadsheet part numbers & descriptions in a single pass
spreadsheet_data = {}
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    part_number = normalize_part_number(str(row[part_number_col - 1].value))
    spare_part = str(row[spare_part_col - 1].value).strip()
    spreadsheet_data[part_number] = spare_part

print(f"\n Extracted {len(spreadsheet_data)} spreadsheet part numbers.")

# Identify unmatched part numbers
supersedes_matches = set(spreadsheet_data) - manual_part_numbers

# Similarity matching
final_supersedes = {}
for sup in supersedes_matches:
    close_matches = get_close_matches(sup, manual_part_numbers, n=1, cutoff=similarity_threshold)
    if close_matches:
        final_supersedes[sup] = close_matches[0]

if final_supersedes:
    print("\n Highly Similar Superseded Part Numbers Found:")
    for new, old in final_supersedes.items():
        print(f" {new} → Likely Supersedes → {old}")

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        part_number = normalize_part_number(str(row[part_number_col - 1].value))
        machine_cell = row[machine_name_col - 1]
        if part_number in manual_part_numbers or part_number in final_supersedes:
            machine_cell.value = f"{machine_cell.value} & {machine_name}" if machine_cell.value else machine_name
            machine_cell.font = Font(color="FF0000")  # Red for machine names

# Save the updated workbook
wb.save(output_path)
print(f"\n Updated Excel file saved as: {output_path}")


 Processing: /content/SPC-2531134-0108.pdf

 Extracted 2048 normalized part numbers.
 Sample extracted parts: ['90458-1740', '321101-0233', '1556532-0000', '1556709-0100', '1258584-0200', '90232-0128', '353243-0007', '191796', '2547726-0100', '1497331-0000']

 Extracted 945 spreadsheet part numbers.

 Highly Similar Superseded Part Numbers Found:
 441721-0000 → Likely Supersedes → 479261-0000
 90230-17 → Likely Supersedes → 90237-0175
 31015308050 → Likely Supersedes → 308050
 62022 → Likely Supersedes → 662202-2
 5469290000 → Likely Supersedes → 548599-0000
 5392120000 → Likely Supersedes → 532412-0000
 90214-0138 → Likely Supersedes → 90214-0213
 90252-0041 → Likely Supersedes → 90258-0046
 1021090-0000 → Likely Supersedes → 1021990-0000
 90042273 → Likely Supersedes → 90042-0271
 10653410000 → Likely Supersedes → 2616531-0000
 90531-2169 → Likely Supersedes → 90458-2916
 3152020612 → Likely Supersedes → 315202-0222
 1074229 → Likely Supersedes → 1074127-2
 1366750 → Likely Supersed

In [ ]:
import fitz  # PyMuPDF (Efficient text extraction)
import pytesseract
from pdf2image import convert_from_path  # OCR (fallback)
import pandas as pd
import re
from openpyxl import load_workbook
import glob
from difflib import get_close_matches
from openpyxl.styles import Font

#  PARAMETERS
pdf_paths = glob.glob("/content/*.pdf")  # Load all PDFs in directory
excel_path = "/content/Updated_Spare_Parts 5.xlsx"
output_path = "/content/Updated_Spare_Parts.xlsx"
machine_name = "TBA19"
similarity_threshold = 0.7  # 70% similarity required for matching

# Load Excel file (keeps styles, fonts, colors)
wb = load_workbook(excel_path)
ws = wb["Spares"]

# Find column indexes
header_row = [cell.value for cell in ws[1]]
part_number_col = header_row.index("Part Number") + 1
machine_name_col = header_row.index("Machine Name") + 1
spare_part_col = header_row.index("Spare Part") + 1

# Normalize part numbers
def normalize_part_number(part):
    if not part:
        return ""
    part = part.upper().strip()
    part = re.sub(r"^[A-C]-", "", part)  # Remove "A-", "B-", "C-" prefix
    part = re.sub(r"[^\d-]", "", part)  # Keep only numbers and dashes
    return part

# Extract text efficiently using PyMuPDF (fitz)
def extract_text_pymupdf(pdf_path):
    return "\n".join([page.get_text("text") for page in fitz.open(pdf_path)])

# OCR extraction (only if PyMuPDF fails)
def extract_text_ocr(pdf_path):
    return "\n".join([pytesseract.image_to_string(img) for img in convert_from_path(pdf_path)])

# Extract and normalize part numbers from text
def extract_part_numbers(text):
    raw_parts = re.findall(r"[A-Z]?-?\d{5,10}-?\d{1,4}", text)
    return {normalize_part_number(part) for part in raw_parts}

# Extract part numbers from manuals
manual_part_numbers = set()
for pdf_path in pdf_paths:
    print(f"\n Processing: {pdf_path}")
    extracted_text = extract_text_pymupdf(pdf_path) or extract_text_ocr(pdf_path)
    manual_part_numbers.update(extract_part_numbers(extracted_text))

print(f"\n Extracted {len(manual_part_numbers)} normalized part numbers.")
print(" Sample extracted parts:", list(manual_part_numbers)[:10])

# Load spreadsheet part numbers & descriptions in a single pass
spreadsheet_data = {}
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    part_number = normalize_part_number(str(row[part_number_col - 1].value))
    spare_part = str(row[spare_part_col - 1].value).strip()
    spreadsheet_data[part_number] = spare_part

print(f"\n Extracted {len(spreadsheet_data)} spreadsheet part numbers.")

# Identify unmatched part numbers
supersedes_matches = set(spreadsheet_data) - manual_part_numbers

# Similarity matching
final_supersedes = {}
for sup in supersedes_matches:
    close_matches = get_close_matches(sup, manual_part_numbers, n=1, cutoff=similarity_threshold)
    if close_matches:
        final_supersedes[sup] = close_matches[0]

if final_supersedes:
    print("\n Highly Similar Superseded Part Numbers Found:")
    for new, old in final_supersedes.items():
        print(f" {new} → Likely Supersedes → {old}")

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        part_number = normalize_part_number(str(row[part_number_col - 1].value))
        machine_cell = row[machine_name_col - 1]
        if (part_number in manual_part_numbers or part_number in final_supersedes) and not machine_cell.value:
            machine_cell.value = machine_name
            machine_cell.font = Font(color="FF0000")  # Red for machine names

# Save the updated workbook
wb.save(output_path)
print(f"\n Updated Excel file saved as: {output_path}")



 Processing: /content/SPC-2771717-0106.pdf

 Extracted 6940 normalized part numbers.
 Sample extracted parts: ['90005-0143', '1350148-1', '2576350-0000', '256181-0000', '272033-0000', '90459-2425', '321101-0233', '2570400-0000', '572158-0000', '312101-0376']

 Extracted 945 spreadsheet part numbers.

 Highly Similar Superseded Part Numbers Found:
 441721-0000 → Likely Supersedes → 441871-0000
 90230-17 → Likely Supersedes → 90270-0017
 31015308050 → Likely Supersedes → 90153-0050
 2729861 → Likely Supersedes → 272986-0001
 5469290000 → Likely Supersedes → 546929-0000
 5392120000 → Likely Supersedes → 539212-0000
 90252-0041 → Likely Supersedes → 90425-0016
 1021090-0000 → Likely Supersedes → 2514090-0000
 90042273 → Likely Supersedes → 90042-0173
 10653410000 → Likely Supersedes → 1523451-0000
 90531-2169 → Likely Supersedes → 90459-2216
 3152020612 → Likely Supersedes → 315202-0120
 1074229 → Likely Supersedes → 180722
 1366750 → Likely Supersedes → 367950
 2547954 → Likely Supersede

In [ ]:
!apt-get install tesseract-ocr -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (12.1 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 126239 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e